In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
import torch
import numpy as np

def data_sequencer(data: torch.Tensor, sequence_len: int, device="cpu"):
    xs = []
    ys = []

    for i in range(len(data) - sequence_len):
        xs.append(data[i: i+sequence_len])
        ys.append(data[i+sequence_len])

    xs = np.array(xs)
    ys = np.array(ys)

    xs = torch.tensor(xs, dtype=torch.float32, device=device)
    ys = torch.tensor(ys, dtype=torch.float32, device=device)

    return xs, ys


In [3]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

def data_fragmenter(df: pd.DataFrame, device="cpu"):
    scaler = MinMaxScaler()

    df_fundamental = df[["Open", "High", "Low", "Close", "Volume"]]
    df_mavg = df[["EMA_12", "EMA_26"]]
    df_mi = df[["MACD", "Signal", "RSI", "CCI"]]
    df_adx = df[["ADX"]]

    # normalize
    df_fundamental = scaler.fit_transform(df_fundamental)
    df_mavg = scaler.fit_transform(df_mavg)
    df_mi = scaler.fit_transform(df_mi)
    df_adx = scaler.fit_transform(df_adx)

    fundamental_x, fundamental_y = data_sequencer(df_fundamental, 30, device=device)
    mavg_x, mavg_y = data_sequencer(df_mavg, 30, device=device)
    mi_x, mi_y = data_sequencer(df_mi, 30, device=device)
    adx_x, adx_y = data_sequencer(df_adx, 30, device=device)

    return fundamental_x, fundamental_y, mavg_x, mavg_y, mi_x, mi_y, adx_x, adx_y


df = pd.read_csv("./stocks_data/AAPL_data.csv")
fundamental_x, fundamental_y, mavg_x, mavg_y, mi_x, mi_y, adx_x, adx_y = data_fragmenter(df=df, device=device)


In [4]:
## Make LSTM that can adapt to shape of data
import torch
from torch import nn

class LSTMv1(nn.Module):
    def __init__(self, input_features, hidden_features, num_layers, output_features, device="cpu"):
        super().__init__()
        self.hidden_features = hidden_features
        self.num_layers = num_layers
        self.device = device

        self.lstm = nn.LSTM(input_features, hidden_features, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_features, output_features)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features, device=self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_features, device=self.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out[:, -1, :]


In [5]:
# INPUT_FEATURES = len(X_train[0, 0, :])
# HIDDEN_FEATURES = 30
# NUM_LAYERS = 3
# OUTPUT_FEATURES = len(y_train[0, :])

model_fundamental = LSTMv1(len(fundamental_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(fundamental_y[0, :]), device=device)
model_mavg = LSTMv1(len(mavg_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(mavg_y[0, :]), device=device)
model_mi = LSTMv1(len(mi_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(mi_y[0, :]), device=device)
model_adx = LSTMv1(len(adx_x[0, 0, :]), hidden_features=30, num_layers=3, output_features=len(adx_y[0, :]), device=device)


In [6]:
loss_fn = nn.MSELoss()
loss_fn.to(device)
optimizer_fundamental = torch.optim.Adam(params=model_fundamental.parameters(), lr=0.001)
optimizer_mavg = torch.optim.Adam(params=model_mavg.parameters(), lr=0.001)
optimizer_mi = torch.optim.Adam(params=model_mi.parameters(), lr=0.001)
optimizer_adx = torch.optim.Adam(params=model_adx.parameters(), lr=0.001)